# This notebook contains training step and analysis using nmae result...

In [11]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns

from pandas.plotting import autocorrelation_plot
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression, GenericUnivariateSelect
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score, KFold
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


In [3]:
# Load write dataset t100
x_t100 = pd.read_csv('datasets/exp90c_11h/t100/prometheus_metrics_wide.csv', low_memory=True).apply(pd.to_numeric, errors='coerce').fillna(0)
print(x_t100.shape)
# x_t100 = x_t100.iloc[3600:]
print(x_t100.shape)
x_t100.head(5)

(40203, 1814)
(40203, 1814)


,timestamp,container_blkio_device_usage_total_0,container_blkio_device_usage_total_1,container_blkio_device_usage_total_2,container_blkio_device_usage_total_3,container_blkio_device_usage_total_4,container_blkio_device_usage_total_5,container_blkio_device_usage_total_6,container_blkio_device_usage_total_7,container_blkio_device_usage_total_8,...,network_transmit_bytes_per_container_30,network_transmit_bytes_per_container_31,network_transmit_bytes_per_container_32,network_transmit_bytes_per_container_33,network_transmit_bytes_per_container_34,network_transmit_bytes_per_container_35,network_transmit_bytes_per_container_36,network_transmit_bytes_per_container_37,network_transmit_bytes_per_container_38,network_transmit_bytes_per_container_39
0,1764340528,8192,24154112,8192,19644416,15339520,59719680,15339520,44359680,0,...,35410.262106,0,11039.204213,0,36037.730154,0,14388.803681,0,0.0,0.0
1,1764340529,8192,24154112,8192,19644416,15339520,59719680,15339520,44359680,0,...,34625.461655,0,11847.457627,0,36037.730154,0,14540.320706,0,0.0,0.0
2,1764340530,8192,24166400,8192,19656704,15339520,59719680,15339520,44359680,0,...,41155.647383,0,14373.485714,0,38306.829112,0,11278.554996,0,0.0,0.0
3,1764340531,8192,24166400,8192,19656704,15339520,59719680,15339520,44359680,0,...,36411.644843,0,12909.598206,0,47245.251662,0,14154.909707,0,0.0,0.0
4,1764340532,8192,24166400,8192,19656704,15339520,59719680,15339520,44359680,0,...,34898.272717,0,12500.372856,0,43470.363289,0,12524.916944,0,0.0,0.0


In [4]:
# Load y t100
y_t100 = pd.read_csv('datasets/exp90c_11h/t100/20251128_143528564_w.csv', low_memory=True).apply(pd.to_numeric, errors='coerce').fillna(0)
print(y_t100.shape)
# y_t100 = y_t100.iloc[3600:]
print(y_t100.shape)
y_t100.head(5)

(40203, 45)
(40203, 45)


,timestamp,queries_num,queries_requested,errors_occurred,iter_errors_occurred,average_latency,99_9_latency_percentile,mean_rate,one_minute_rate,five_minute_rate,...,w_min,w_max,w_mean,w_std_dev,w_median,w_75th_percentile,w_95th_percentile,w_98th_percentile,w_99th_percentile,w_99_9th_percentile
0,1764340528,0,0,0,0,0,0,0.000000,0.0,0.0,...,0,0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0
1,1764340529,9,0,0,0,15,1,0.059516,0.0,0.0,...,4,49,15.666667,19.052559,6.0,24.333333,49.0,49.0,49.0,49
2,1764340530,16,0,0,0,11,1,0.105110,0.0,0.0,...,4,49,11.375000,14.867750,5.0,9.333333,49.0,49.0,49.0,49
3,1764340531,24,0,0,0,9,1,0.156637,0.0,0.0,...,4,49,9.166667,12.426013,5.0,6.000000,49.0,49.0,49.0,49
4,1764340532,31,0,0,0,7,1,0.201011,0.0,0.0,...,4,49,8.032258,11.088985,5.0,5.000000,49.0,49.0,49.0,49


In [ ]:
# Merge features and taget dataframe...

